# Overview: Gradient Boosting

### Introduction

Our goal is to find a function $F^*(\mathbf{x})$ that maps $x$ to $y$ such that loss function $\Psi(y, F(\mathbf{x}))$ is minimized over the joint distribution of all $(y, x)$ values

$$F^*(\mathbf{x}) = \arg \min_{F(\mathbf{x})} E_{y, \mathbf{x}} \, \Psi(y, F(\mathbf{x}))$$


Boosting: additive expansion with base learner $h(\mathbf{x}; \mathbf{a}_m)$
$$F(\mathbf{x}) = \sum_{m=0}^{M} \beta_m h(\mathbf{x}; \mathbf{a}_m)$$


### Stochastic Gradient Boosting

$$F_0(\mathbf{x}) = \arg \min_{\gamma} \sum_{i=1}^{N} \Psi(y_i, \gamma)$$


**Stochastic Gradient Tree Boosting Pseudocode**
- Init input
    - loss_func = squared_error
    - learning_rate = 0.1
    - n_esitmators = 100                         
        - this is denoted as M in the training pseudocode below
    - subsample = 0.6                              
        - fraction for subsampling, if subsample<1, it is stochastic
    - criterion_func = friedman_mse             
        - the function to measure the quality of a split
    - max_depth = 3                              
        - the maximum depth for each tree
    - random_state = None                        
        - controls batch, and ....?

- Train(X, Y)
    - start with an initial guess, say F(x) = mean(Y)
    - for m = 1 to M do:
        - X_batch, Y_batch = random_sampling(X, Y, fraction for subsampling (batch size))    # fraction < 1 leads to stochastic gradient boosting
            - question: do we need to remove the batch from the whole pool at each iteration?
        - Y_residual = negative gradient of loss between (true) Y and current F based on the batch
        - weak learner h(x) = new weakLearner.fit(X_batch, Y_residual)
        - weak learner Tree $\{R_{lm}\}_1^L$ = L-terminal node tree fitting on (X_batch, Y_residual)
            - here $\{R_{lm}\}_1^L$ represents a set of L regions (leaf nodes) in stage m
            - $R_{lm}$ represents the l-th region (leaf node) in stage m
        - $\gamma_{lm} = arg min_{\gamma} \sum_{x-batch \in R_{lm}}$ loss(Y_batch, F(x)+$\gamma$)
            - that is, for each region l at stage m, find $\gamma$ such that it minimizes the loss for the new F(x) = F(x) + $\gamma$
        - update F(x) = F(x) + learning_rate * $\gamma * 1(x \in R_{lm})$
            - that is, F(x) = F(x) + learning_rate * new tree
            - note here we are multiplying the new tree with learning rate (different from how we obtain the best $\gamma$)
    - end for

- Predict(X)
    - TODO

- Loss
    - TODO